In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm, colors
import glob

from numba import njit, vectorize, float32

from typing import Callable, Optional, Union

import hoomd
import gsd.hoomd
import freud

import sys
import time
import pickle
import gc
import warnings
import copy
import pathlib

import os
import sys

import signac

from dataclasses import dataclass
from dataclasses_json import dataclass_json

In [2]:
from monk import nb, prep, pair, render, utils, workflow, grid

config = workflow.get_config()

/home/ian/Projects/work/monk/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
project: signac.Project = signac.init_project(name="mlj-production", root=config['root'])
project.doc

{'ref_sim_temps': [0.41, 0.43, 0.45, 0.47, 0.49, 0.51, 0.53, 0.55, 0.58, 0.61, 0.64, 0.67, 0.7, 0.75, 0.8, 0.9, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0], 'avail_seed': 18, 'alpha_iters': 40, 'dt': 0.001, 'usteps': 1000}

In [3]:
from logging import warning


def mlj_alpha_est(T, phi, delta, min_clamp=1.0):
    "Estimate alpha relaxation time for a given MLJ statepoint"
    params = np.array([ 0.49717018, -1.0956471 ,  2.54414117, -2.15685159])
    x = phi**(2/3)/(T*(1 - delta*2**(1/6))**2)
    if x > 3.0:
        msg = f"Alpha estimate is beyond the original fit range. inv_T* = {x}"
        warnings.warn(msg)
    x = np.exp(np.polyval(params, x))
    return np.clip(x, min_clamp, np.inf)


def iso_mlj_pred(T, phi, delta):
    """Find temperature with equivalent dynamics to ref MLJ system (delta=0.0, phi=1.2)"""
    return T * (phi/1.2)**(2/3) / (1 - delta*2**(1/6))**2

In [4]:
mlj_alpha_est(0.41, 1.2, 0.0)

1018.9213055070264

In [8]:
iso_mlj_pred(0.41, 1.1, 0.5)

2.0096464413065416

In [27]:
ref_sim_temps = np.array([0.41, 0.43, 0.45, 0.47, 0.49, 0.51, 0.53, 0.55, 0.58, 0.61, 0.64, 0.67, 0.7, 0.75, 0.8, 0.9, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0])

In [16]:
project.doc["dt"] = 0.001
project.doc["usteps"] = 1_000

In [48]:
project.doc["ref_sim_temps"] = list(ref_sim_temps)
project.doc["avail_seed"] = 0

/home/ian/Projects/work/monk/.venv/lib/python3.10/site-packages/signac/synced_collections/numpy_utils.py:46: NumpyConversionWarning: Any numpy types provided will be transparently converted to the closest base Python equivalents.
  warnings.warn(NUMPY_CONVERSION_WARNING, NumpyConversionWarning)


In [49]:
N = 2**15  # 32758
deltas = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
phis = [1.2, 1.18, 1.16, 1.14, 1.12, 1.1]

i = 0
for replica in range(3):
    for delta, phi in zip(deltas, phis):
        sp = {"N": N, "replica": replica, "delta": delta, "phi": phi}
        job = project.open_job(sp)
        if "init" not in job.doc:
            job.init()
            job.doc["init"] = True
            job.doc["seed"] = project.doc["avail_seed"]
            project.doc["avail_seed"] += 1

In [50]:
project.doc["avail_seed"]

18

In [35]:
# Everything looks good!

for job in project:
    print(job.sp, job.doc)
    temps = project.doc["ref_sim_temps"]
    print("[", end="")
    new_temps = np.around(iso_mlj_pred(np.array(temps), job.sp["phi"], job.sp["delta"]), decimals=2)
    for t in new_temps[:-1]:
        print(f"{t:.2f}, ", end="")
    # print(f"{new_temps[-1]:.3f}]")

    job.doc["temps"] = new_temps[::-1]  # store temps in the order that they will be run
    job.doc["alpha_ests"] = [mlj_alpha_est(t, job.sp["phi"], job.sp["delta"]) for t in new_temps[::-1]]
    # print(job.doc["temps"])

{'N': 32768, 'replica': 0, 'delta': 0.4, 'phi': 1.12} {'seed': 4, 'init': True, 'temps': [6.29, 5.66, 5.03, 4.4, 3.77, 3.15, 2.83, 2.52, 2.36, 2.2, 2.11, 2.01, 1.92, 1.82, 1.73, 1.67, 1.6, 1.54, 1.48, 1.42, 1.35, 1.29], 'alpha_ests': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0322382060839836, 1.3410428930764626, 1.9055124972161805, 2.4242655033016964, 3.2781409371501735, 4.026881451885778, 5.266453529750067, 7.014891277639196, 10.290238479861296, 15.663961532701151, 21.799547231262366, 34.16576826659126, 53.69068129565428, 91.05173912026156, 169.59829552055575, 406.8345114446819, 1013.4550042854213], 'checkpoint': {'temp_idx': 0, 'start_tstep': 100000}}
[1.29, 1.35, 1.42, 1.48, 1.54, 1.60, 1.67, 1.73, 1.82, 1.92, 2.01, 2.11, 2.20, 2.36, 2.52, 2.83, 3.15, 3.77, 4.40, 5.03, 5.66, {'N': 32768, 'replica': 0, 'delta': 0.1, 'phi': 1.18} {'seed': 1, 'init': True, 'temps': [2.51, 2.26, 2.01, 1.76, 1.51, 1.25, 1.13, 1.0, 0.94, 0.88, 0.84, 0.8, 0.77, 0.73, 0.69, 0.67, 0.64, 0.61, 0.59, 0.56, 0.54, 0.51], 'alp

In [11]:
# generate initial states

for job in project:

    if job.isfile('init.gsd'):
        continue
    
    doc = job.doc
    sp = job.sp

    sim = prep.quick_sim(sp["N"], sp["phi"], hoomd.device.GPU(), ratios=[80, 20], diams=[1.0, 0.88], seed=doc["seed"])

    integrator = hoomd.md.Integrator(dt=0.001)
    tree = hoomd.md.nlist.Tree(0.3)
    mlj = pair.KA_ModLJ(tree, sp["delta"])

    start_temp = doc["temps"][0]
    print(sp, start_temp)
    nvt = hoomd.md.methods.NVT(hoomd.filter.All(), start_temp, 0.1)
    integrator.forces = [mlj]
    integrator.methods = [nvt]

    sim.operations.integrator = integrator
    
    sim.state.thermalize_particle_momenta(hoomd.filter.All(), start_temp)

    sim.run(0)
    nvt.thermalize_thermostat_dof()

    sim.run(100_000)  # do a moderate equilibration

    hoomd.write.GSD.write(sim.state, job.fn("init.gsd"))

    del sim


{'N': 32768, 'replica': 0, 'delta': 0.4, 'phi': 1.12} 6.29
{'N': 32768, 'replica': 0, 'delta': 0.1, 'phi': 1.18} 2.51
{'N': 32768, 'replica': 1, 'delta': 0.3, 'phi': 1.14} 4.39
{'N': 32768, 'replica': 0, 'delta': 0.3, 'phi': 1.14} 4.39
{'N': 32768, 'replica': 1, 'delta': 0.2, 'phi': 1.16} 3.25
{'N': 32768, 'replica': 1, 'delta': 0.1, 'phi': 1.18} 2.51
{'N': 32768, 'replica': 1, 'delta': 0.0, 'phi': 1.2} 2.0
{'N': 32768, 'replica': 2, 'delta': 0.3, 'phi': 1.14} 4.39
{'N': 32768, 'replica': 2, 'delta': 0.4, 'phi': 1.12} 6.29
{'N': 32768, 'replica': 2, 'delta': 0.2, 'phi': 1.16} 3.25
{'N': 32768, 'replica': 2, 'delta': 0.0, 'phi': 1.2} 2.0
{'N': 32768, 'replica': 0, 'delta': 0.5, 'phi': 1.1} 9.8
{'N': 32768, 'replica': 0, 'delta': 0.0, 'phi': 1.2} 2.0
{'N': 32768, 'replica': 2, 'delta': 0.5, 'phi': 1.1} 9.8
{'N': 32768, 'replica': 2, 'delta': 0.1, 'phi': 1.18} 2.51
{'N': 32768, 'replica': 1, 'delta': 0.5, 'phi': 1.1} 9.8
{'N': 32768, 'replica': 0, 'delta': 0.2, 'phi': 1.16} 3.25
{'N': 327

In [6]:
hoomd.device.GPU.get_available_devices()

['[0]NVIDIA GeForce RTX 2080  46 SM_7.5 @ 1.71 GHz, 7979 MiB DRAM']

# Primary equilibration

In [11]:
RESTART = False
OMIT = -2

In [12]:
# now go backwards in temperature and run simulations

@dataclass_json
@dataclass
class Checkpoint:
    temp_idx: int
    start_tstep: int

# sys.exit()
for job_idx, job in enumerate(project):

    assert isinstance(job, signac.Project.Job)

    # get job parameters
    sp = job.sp
    doc = job.doc

    print(sp)

    # start simulation
    sim = hoomd.Simulation(hoomd.device.GPU(), seed=doc["seed"])

    # if restart file is present, initialize from that
    if job.isfile('restart.gsd') and "checkpoint" in doc and not RESTART:
        cp = Checkpoint.from_dict(doc["checkpoint"])
        if job.isfile("CORRUPTED"):
            print("Job is corrupted! Restarting from last OK state.")
            temp_idx = cp.temp_idx
            if temp_idx == 0:
                restart = job.fn("init.gsd")
            else:
                last_temp = doc["temps"][temp_idx - 1]
                restart = job.fn(f"runs/temp-{last_temp:.2f}/traj.gsd")
            
            sim.create_state_from_gsd(filename=restart)
        else:
            sim.create_state_from_gsd(filename=job.fn('restart.gsd'))
        cp_temp = doc["temps"][cp.temp_idx]
        print("Checkpoint temp:", cp_temp)
        print("Resuming from timestep:", sim.timestep)
    else:
        sim.create_state_from_gsd(filename=job.fn('init.gsd'))
        cp = Checkpoint(0, sim.timestep)

    integrator = hoomd.md.Integrator(dt=project.doc["dt"])
    tree = hoomd.md.nlist.Tree(0.3)
    mlj = pair.KA_ModLJ(tree, sp["delta"])

    nvt = hoomd.md.methods.NVT(hoomd.filter.All(), doc["temps"][cp.temp_idx], 0.1)
    integrator.forces = [mlj]
    integrator.methods = [nvt]

    sim.operations.integrator = integrator

    print("Running up to temp:", doc["temps"][OMIT-1])

    for temp in doc["temps"][cp.temp_idx:OMIT]:

        print("Current temp:", temp)

        nvt.kT = temp  # ensure the temperature is correctly set

        out_period = int(np.around(doc["alpha_ests"][cp.temp_idx] * project.doc["usteps"] / 100))

        log = hoomd.logging.Logger()
        log.add(sim, quantities=["timestep"])

        out_path = f"runs/temp-{temp:.2f}"
        os.makedirs(job.fn(out_path), exist_ok=True)

        # Write `trajectory.gsd` in append mode.
        gsd_writer = hoomd.write.GSD(filename=job.fn(f'{out_path}/traj.gsd'),
                                    trigger=hoomd.trigger.Periodic(out_period, phase=cp.start_tstep),
                                    mode='ab',
                                    log=log,
                                    )
        sim.operations.writers.append(gsd_writer)
        
        end_step = int(np.around(project.doc["alpha_iters"] * doc["alpha_ests"][cp.temp_idx] * project.doc["usteps"])) + cp.start_tstep

        print(temp, cp, end_step)
            
        try:
            # Loop until the simulation reaches the target timestep.
            while sim.timestep < end_step:
                # Run the simulation in chunks of 10,000 time steps.
                sim.run(min(100_000, end_step - sim.timestep))

            sim.operations.writers.clear() # clear the writers

            # Once we pass the while loop, we are ready to increment the
            # checkpoint data to the next temperature.

            # Copy checkpoint data so we can reassign it atomically
            cp_copy = copy.deepcopy(cp)
            cp_copy.temp_idx += 1
            cp_copy.start_tstep = sim.timestep
            cp = cp_copy  # reassign our copy as the new checkpoint

        finally:
            # Write the state of the system to `restart.gsd`.

            print("Writing restart file at timestep:", sim.timestep)

            hoomd.write.GSD.write(state=sim.state,
                                mode='wb',
                                filename=job.fn('restart.gsd'))
            
            job.doc["checkpoint"] = cp.to_dict()
    print("Job complete!")
    del sim

{'N': 32768, 'replica': 0, 'delta': 0.4, 'phi': 1.12}
Checkpoint temp: 1.35
Resuming from timestep: 17201988
Running up to temp: 1.42
Job complete!
{'N': 32768, 'replica': 0, 'delta': 0.1, 'phi': 1.18}
Checkpoint temp: 0.54
Resuming from timestep: 18705061
Running up to temp: 0.56
Job complete!
{'N': 32768, 'replica': 1, 'delta': 0.3, 'phi': 1.14}
Checkpoint temp: 0.94
Resuming from timestep: 17465993
Running up to temp: 0.99
Job complete!
{'N': 32768, 'replica': 0, 'delta': 0.3, 'phi': 1.14}
Checkpoint temp: 0.94
Resuming from timestep: 17465993
Running up to temp: 0.99
Job complete!
{'N': 32768, 'replica': 1, 'delta': 0.2, 'phi': 1.16}
Checkpoint temp: 0.7
Resuming from timestep: 18017500
Running up to temp: 0.73
Job complete!
{'N': 32768, 'replica': 1, 'delta': 0.1, 'phi': 1.18}
Checkpoint temp: 0.54
Resuming from timestep: 18705061
Running up to temp: 0.56
Job complete!
{'N': 32768, 'replica': 1, 'delta': 0.0, 'phi': 1.2}
Checkpoint temp: 0.43
Resuming from timestep: 17557689
Runni

# Softness & Excess Entropy data run

Run each system for $100 \ t_{LJ}$ to get some simulation data to analyze

Run the systems that we'll use to compute softness for $10,000 \ t_{LJ}$

In [4]:
for job in project.find_jobs(None, {"_CRYSTAL": {"$exists": False}}):
    print("Job", job.sp)

    sp = job.sp
    doc = job.doc
    
    assert isinstance(job, signac.Project.Job)

    runs = sorted(glob.glob(job.fn("runs/temp-*/traj.gsd")))

    for run in runs:

        temp = utils.extract_between(run, "temp-", "/traj.gsd")
        out_dir = f"short_runs/temp-{temp}"
        os.makedirs(job.fn(out_dir), exist_ok=True)
        done = f"{out_dir}/DONE"
        if job.isfile(done):
            print("\tTemp is done!", temp)
            continue

        print("\tTemp:", temp)

        sim = hoomd.Simulation(hoomd.device.GPU(), seed=doc["seed"])
        sim.create_state_from_gsd(filename=run)

        integrator = hoomd.md.Integrator(dt=project.doc["dt"])
        tree = hoomd.md.nlist.Tree(0.3)
        mlj = pair.KA_ModLJ(tree, sp["delta"])

        nvt = hoomd.md.methods.NVT(hoomd.filter.All(), float(temp), 0.1)
        integrator.forces = [mlj]
        integrator.methods = [nvt]

        sim.operations.integrator = integrator

        log = hoomd.logging.Logger()
        log.add(sim, quantities=["timestep"])

        sim.run(0)

        usteps = project.doc["usteps"]

        gsd_writer = hoomd.write.GSD(filename=job.fn(f'{out_dir}/traj.gsd'),
                                    trigger=hoomd.trigger.Periodic(usteps, phase=sim.timestep),
                                    mode='wb',
                                    log=log,
                                    )
        sim.operations.writers.append(gsd_writer)

        sim.run(usteps * 100)

        pathlib.Path(job.fn(done)).touch()
        print("\tDone!", temp)
        
        del mlj, tree, nvt, integrator, log, sim
        gc.collect()

Job {'N': 32768, 'replica': 2, 'delta': 0.3, 'phi': 1.14}
	Temp is done! 0.99
	Temp is done! 1.03
	Temp is done! 1.08
	Temp is done! 1.12
	Temp is done! 1.16
	Temp is done! 1.21
	Temp is done! 1.27
	Temp is done! 1.34
	Temp is done! 1.41
	Temp is done! 1.47
	Temp is done! 1.54
	Temp is done! 1.65
	Temp is done! 1.76
	Temp is done! 1.98
	Temp is done! 2.20
	Temp is done! 2.64
	Temp is done! 3.08
	Temp is done! 3.51
	Temp is done! 3.95
	Temp is done! 4.39
Job {'N': 32768, 'replica': 1, 'delta': 0.2, 'phi': 1.16}
	Temp is done! 0.73
	Temp is done! 0.76
	Temp is done! 0.80
	Temp is done! 0.83
	Temp is done! 0.86
	Temp is done! 0.89
	Temp is done! 0.94
	Temp is done! 0.99
	Temp is done! 1.04
	Temp is done! 1.09
	Temp is done! 1.14
	Temp is done! 1.22
	Temp is done! 1.30
	Temp is done! 1.46
	Temp is done! 1.63
	Temp is done! 1.95
	Temp is done! 2.28
	Temp is done! 2.60
	Temp is done! 2.93
	Temp is done! 3.25
Job {'N': 32768, 'replica': 0, 'delta': 0.1, 'phi': 1.18}
	Temp is done! 0.56
	Temp 

# Secondary equilibration

Estimates for the alpha relaxation times for some of the simulation are not quite right

In [5]:
for job in project.find_jobs(None, {"_CRYSTAL": {"$exists": False}}):
    print("Job", job.sp)

    sp = job.sp
    doc = job.doc
    
    assert isinstance(job, signac.Project.Job)

    runs = sorted(glob.glob(job.fn("short_runs/temp-*/traj.gsd")))

    for run in runs[1:2]:

        temp = utils.extract_between(run, "temp-", "/traj.gsd")
        out_dir = f"training_runs/temp-{temp}"
        os.makedirs(job.fn(out_dir), exist_ok=True)
        done = f"{out_dir}/DONE"
        if job.isfile(done):
            print("\tTemp is done!", temp)
            continue

        print("\tTemp:", temp)

        sim = hoomd.Simulation(hoomd.device.GPU(), seed=doc["seed"])
        sim.create_state_from_gsd(filename=run)

        integrator = hoomd.md.Integrator(dt=project.doc["dt"])
        tree = hoomd.md.nlist.Tree(0.3)
        mlj = pair.KA_ModLJ(tree, sp["delta"])

        nvt = hoomd.md.methods.NVT(hoomd.filter.All(), float(temp), 0.1)
        integrator.forces = [mlj]
        integrator.methods = [nvt]

        sim.operations.integrator = integrator

        log = hoomd.logging.Logger()
        log.add(sim, quantities=["timestep"])

        sim.run(0)

        usteps = project.doc["usteps"]

        gsd_writer = hoomd.write.GSD(filename=job.fn(f'{out_dir}/traj.gsd'),
                                    trigger=hoomd.trigger.Periodic(usteps, phase=sim.timestep),
                                    mode='wb',
                                    log=log,
                                    )
        sim.operations.writers.append(gsd_writer)

        sim.run(usteps * 1000)

        pathlib.Path(job.fn(done)).touch()
        print("\tDone!", temp)
        
        del mlj, tree, nvt, integrator, log, sim
        gc.collect()

Job {'N': 32768, 'replica': 2, 'delta': 0.3, 'phi': 1.14}
	Temp: 1.03
	Done! 1.03
Job {'N': 32768, 'replica': 1, 'delta': 0.2, 'phi': 1.16}
	Temp: 0.76
	Done! 0.76
Job {'N': 32768, 'replica': 0, 'delta': 0.1, 'phi': 1.18}
	Temp: 0.59
	Done! 0.59
Job {'N': 32768, 'replica': 2, 'delta': 0.4, 'phi': 1.12}
	Temp: 1.48
	Done! 1.48
Job {'N': 32768, 'replica': 0, 'delta': 0.0, 'phi': 1.2}
	Temp: 0.47
	Done! 0.47
Job {'N': 32768, 'replica': 1, 'delta': 0.3, 'phi': 1.14}
	Temp: 1.03
	Done! 1.03
Job {'N': 32768, 'replica': 0, 'delta': 0.4, 'phi': 1.12}
	Temp: 1.48
	Done! 1.48
Job {'N': 32768, 'replica': 2, 'delta': 0.1, 'phi': 1.18}
	Temp: 0.59
	Done! 0.59
Job {'N': 32768, 'replica': 1, 'delta': 0.4, 'phi': 1.12}
	Temp: 1.48
	Done! 1.48
Job {'N': 32768, 'replica': 2, 'delta': 0.0, 'phi': 1.2}
	Temp: 0.47
	Done! 0.47
Job {'N': 32768, 'replica': 0, 'delta': 0.3, 'phi': 1.14}
	Temp: 1.03
	Done! 1.03
Job {'N': 32768, 'replica': 1, 'delta': 0.1, 'phi': 1.18}
	Temp: 0.59
	Done! 0.59
Job {'N': 32768, '